# Lab | Web Scraping

Welcome to the "Books to Scrape" Web Scraping Adventure Lab!

**Objective**

In this lab, we will embark on a mission to unearth valuable insights from the data available on Books to Scrape, an online platform showcasing a wide variety of books. As data analyst, you have been tasked with scraping a specific subset of book data from Books to Scrape to assist publishing companies in understanding the landscape of highly-rated books across different genres. Your insights will help shape future book marketing strategies and publishing decisions.

**Background**

In a world where data has become the new currency, businesses are leveraging big data to make informed decisions that drive success and profitability. The publishing industry, much like others, utilizes data analytics to understand market trends, reader preferences, and the performance of books based on factors such as genre, author, and ratings. Books to Scrape serves as a rich source of such data, offering detailed information about a diverse range of books, making it an ideal platform for extracting insights to aid in informed decision-making within the literary world.

**Task**

Your task is to create a Python script using BeautifulSoup and pandas to scrape Books to Scrape book data, focusing on book ratings and genres. The script should be able to filter books with ratings above a certain threshold and in specific genres. Additionally, the script should structure the scraped data in a tabular format using pandas for further analysis.

**Expected Outcome**

A function named `scrape_books` that takes two parameters: `min_rating` and `max_price`. The function should scrape book data from the "Books to Scrape" website and return a `pandas` DataFrame with the following columns:

**Expected Outcome**

- A function named `scrape_books` that takes two parameters: `min_rating` and `max_price`.
- The function should return a DataFrame with the following columns:
  - **UPC**: The Universal Product Code (UPC) of the book.
  - **Title**: The title of the book.
  - **Price (£)**: The price of the book in pounds.
  - **Rating**: The rating of the book (1-5 stars).
  - **Genre**: The genre of the book.
  - **Availability**: Whether the book is in stock or not.
  - **Description**: A brief description or product description of the book (if available).
  
You will execute this script to scrape data for books with a minimum rating of `4.0 and above` and a maximum price of `£20`. 

Remember to experiment with different ratings and prices to ensure your code is versatile and can handle various searches effectively!

**Resources**

- [Beautiful Soup Documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
- [Pandas Documentation](https://pandas.pydata.org/pandas-docs/stable/index.html)
- [Books to Scrape](https://books.toscrape.com/)


**Hint**

Your first mission is to familiarize yourself with the **Books to Scrape** website. Navigate to [Books to Scrape](http://books.toscrape.com/) and explore the available books to understand their layout and structure. 

Next, think about how you can set parameters for your data extraction:

- **Minimum Rating**: Focus on books with a rating of 4.0 and above.
- **Maximum Price**: Filter for books priced up to £20.

After reviewing the site, you can construct a plan for scraping relevant data. Pay attention to the details displayed for each book, including the title, price, rating, and availability. This will help you identify the correct HTML elements to target with your scraping script.

Make sure to build your scraping URL and logic based on the patterns you observe in the HTML structure of the book listings!


---

**Best of luck! Immerse yourself in the world of books, and may the data be with you!**

**Important Note**:

In the fast-changing online world, websites often update and change their structures. When you try this lab, the **Books to Scrape** website might differ from what you expect.

If you encounter issues due to these changes, like new rules or obstacles preventing data extraction, don’t worry! Get creative.

You can choose another website that interests you and is suitable for scraping data. Options like Wikipedia, The New York Times, or even library databases are great alternatives. The main goal remains the same: extract useful data and enhance your web scraping skills while exploring a source of information you enjoy. This is your opportunity to practice and adapt to different web environments!

In [1]:
# Install / import (usually already installed)
import requests
from bs4 import BeautifulSoup
import pandas as pd
from urllib.parse import urljoin

In [2]:
def scrape_books(min_rating: float, max_price: float) -> pd.DataFrame:
    """
    Scrape Books to Scrape and return a DataFrame filtered by:
      - rating >= min_rating
      - price <= max_price

    Columns returned:
      UPC, Title, Price (£), Rating, Genre, Availability, Description
    """

    BASE_URL = "https://books.toscrape.com/"
    url = BASE_URL  # start at home (page 1)

    # maps the class name on the site to numeric rating
    rating_map = {"One": 1, "Two": 2, "Three": 3, "Four": 4, "Five": 5}

    session = requests.Session()
    session.headers.update({"User-Agent": "Mozilla/5.0 (compatible; webscraping-lab/1.0)"})

    listing_rows = []

    # --- 1) Scrape ALL listing pages (title/price/rating/availability/product_url) ---
    while url:
        res = session.get(url, timeout=30)
        res.raise_for_status()
        soup = BeautifulSoup(res.text, "html.parser")

        books = soup.select("article.product_pod")
        for b in books:
            title = b.h3.a.get("title", "").strip()

            # raw string like "£51.77" sometimes becomes "Â£51.77" depending on encoding
            price_str = b.select_one(".price_color").get_text(strip=True)
            # keep digits and dot only
            price_val = float("".join(ch for ch in price_str if (ch.isdigit() or ch == ".")))

            availability = b.select_one(".availability").get_text(" ", strip=True)

            # rating is encoded in class list: ['star-rating', 'Three']
            rating_classes = b.select_one(".star-rating").get("class", [])
            rating_word = next((c for c in rating_classes if c != "star-rating"), None)
            rating_val = rating_map.get(rating_word, None)

            rel_link = b.h3.a.get("href", "")
            product_url = urljoin(url, rel_link)

            listing_rows.append({
                "Title": title,
                "Price (£)": price_val,
                "Rating": rating_val,
                "Availability": availability,
                "product_url": product_url
            })

        # pagination: follow "next" if exists
        next_btn = soup.select_one("li.next a")
        url = urljoin(url, next_btn["href"]) if next_btn else None

    df = pd.DataFrame(listing_rows)

    # --- 2) Filter by min_rating and max_price (INSIDE the function) ---
    df = df[(df["Rating"] >= float(min_rating)) & (df["Price (£)"] <= float(max_price))].copy()

    # --- 3) Visit each product page to collect UPC, Genre, Description ---
    upcs, genres, descriptions = [], [], []

    for product_url in df["product_url"].tolist():
        r = session.get(product_url, timeout=30)
        r.raise_for_status()
        detail = BeautifulSoup(r.text, "html.parser")

        # UPC: first row in product information table
        upc = None
        table = detail.select_one("table.table.table-striped")
        if table:
            # find the row where th == "UPC"
            for row in table.select("tr"):
                th = row.select_one("th")
                td = row.select_one("td")
                if th and td and th.get_text(strip=True) == "UPC":
                    upc = td.get_text(strip=True)
                    break

        # Genre: breadcrumb usually: Home > Books > <Category> > <Title>
        genre = None
        bc = detail.select("ul.breadcrumb li a")
        # anchors typically include "Home", "Books", then category
        if len(bc) >= 3:
            genre = bc[2].get_text(strip=True)

        # Description: paragraph after #product_description (if exists)
        desc = None
        desc_header = detail.select_one("#product_description")
        if desc_header:
            # next sibling p contains the description
            p = desc_header.find_next_sibling("p")
            if p:
                desc = p.get_text(" ", strip=True)

        upcs.append(upc)
        genres.append(genre)
        descriptions.append(desc)

    df["UPC"] = upcs
    df["Genre"] = genres
    df["Description"] = descriptions

    # final output columns in the exact order requested
    df_final = df[["UPC", "Title", "Price (£)", "Rating", "Genre", "Availability", "Description"]].reset_index(drop=True)
    return df_final

In [3]:
df_result = scrape_books(min_rating=4.0, max_price=20.0)
df_result.head()


,UPC,Title,Price (£),Rating,Genre,Availability,Description
0,ce6396b0f23f6ecc,Set Me Free,17.46,5,Young Adult,In stock,Aaron Ledbetterâs future had been planned ou...
1,6258a1f6a6dcfe50,The Four Agreements: A Practical Guide to Pers...,17.66,5,Spirituality,In stock,"In The Four Agreements, don Miguel Ruiz reveal..."
2,6be3beb0793a53e7,Sophie's World,15.94,5,Philosophy,In stock,A page-turning novel that is also an explorati...
3,657fe5ead67a7767,Untitled Collection: Sabbath Poems 2014,14.27,4,Poetry,In stock,"More than thirty-five years ago, when the weat..."
4,51653ef291ab7ddc,This One Summer,19.49,4,Sequential Art,In stock,"Every summer, Rose goes with her mom and dad t..."


In [6]:
df_result.shape
df_result.columns
df_result["Rating"].min(), df_result["Price (£)"].max()

(4, 19.69)